In [3]:
from DataProcess.utils.filehelper import read_csv_file
from tqdm import tqdm
import csv

goods_same = read_csv_file('./data/goods_same.csv')[1:]
goods_similar = read_csv_file('./data/goods_similar_correct.csv')[1:]

1233383it [00:03, 396392.31it/s]
104171it [00:00, 328142.61it/s]


In [9]:
from DataProcess.utils.filehelper import join_prompt_strings
# 有422个长度超过256

final_data = []
max_length = 0
num_max_length = 0
now_str = ''
id = 0

for data in tqdm(goods_similar):
    if len(data)==7:
        if data[5] == 1:
            pass
        else:
            now_str = join_prompt_strings(prompt="对比相同和相似物资:",str_a = data[3],str_b = data[4])
            if len(now_str)>256:
                num_max_length+=1
            if len(now_str)>max_length:
                max_length = len(now_str)
            else:
                final_data.append([id, now_str,data[5],data[6]])
                id += 1

for data in tqdm(goods_same):
    if len(data) == 4:
        now_str = join_prompt_strings(prompt="对比相同和相似物资:",str_a = data[0],str_b = data[1])
        if len(now_str)>256:
                num_max_length+=1
        if len(now_str)>max_length:
            max_length = len(now_str)
        else:
            final_data.append([id,now_str,data[2],data[3]])
            id += 1
print(num_max_length)

100%|██████████| 1233382/1233382 [00:04<00:00, 265237.95it/s]

422


In [16]:
from sklearn.model_selection import train_test_split

def split_data(data_list, test_size=0.1,stratify=False, random_state=42):
    X = [data[0:2] for data in data_list]
    y = [data[2:] for data in data_list]
    if stratify:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    train_list = [x + list(y) for x, y in zip(X_train, y_train)]
    test_list = [x + list(y) for x, y in zip(X_test, y_test)]
    return train_list, test_list

def export_to_csv(data_list, csv_path):
    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['id','data', 'same', 'similar'])  # 表头
        for i, data in enumerate(data_list):
            writer.writerow(data)

train_data, test_data  =  split_data(final_data, test_size=0.1,stratify=True, random_state=114514)

NameError: name 'final_data' is not defined

In [11]:
export_to_csv(train_data,'./processed/train.csv')
export_to_csv(test_data, './processed/test.csv')

# 抽取部分数据出来人工矫正

In [1]:
# 人工矫正后数据作为参考,调整模型的阈值
from DataProcess.utils.filehelper import read_csv_file
from tqdm import tqdm
import csv

goods_test = read_csv_file('./processed/test.csv')[1:]


133751it [00:00, 351907.32it/s]


In [17]:
import random
def export_to_csv(data_list, csv_path):
    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['id','data', 'same', 'similar'])  # 表头
        for i, data in enumerate(data_list):
            writer.writerow(data)

def extract_balanced_items(items,required_each_count=100):
    # Step 3: Select items
    selected_items = []
    items_00 = []
    items_01 = []
    items_11 = []
    for item in tqdm(items):
        if int(item[2])==0 and int(item[3])==0:
            items_00.append(item)
        if int(item[2])==0 and int(item[3])==1:
            items_01.append(item)
        if int(item[2])==1 and int(item[3])==1:
            items_11.append(item)
    if required_each_count>=len(items):
        return items
    else:
        selected_items.extend(random.sample(items_00, required_each_count))
        selected_items.extend(random.sample(items_01, required_each_count))
        selected_items.extend(random.sample(items_11, required_each_count))
    return selected_items


select_test = extract_balanced_items(goods_test,200)

100%|██████████| 133750/133750 [00:00<00:00, 980973.19it/s]


In [21]:
from operator import itemgetter
processed_data_sorted = sorted(select_test,key=itemgetter(2), reverse=True)
export_to_csv(processed_data_sorted,'./processed/test_sample.csv')
export_to_csv(processed_data_sorted,'./processed/test_sample_correct.csv')

# 对人工矫正后的数据数据分析对比

In [22]:
# 物资a,物资b,拼接后字符,标注标签相同,标注标签相似,矫正标签相同,矫正标签相似,预测标签相同概率,预测标签相似概率,之前的预测标签不相同概率,预测标签相同概率

test_sample = read_csv_file('./processed/test_sample.csv')[1:]
test_sample_correct = read_csv_file('./processed/test_sample_correct.csv')[1:]



601it [00:00, 200252.36it/s]
601it [00:00, 300521.78it/s]


In [24]:


url_predict_new = ''
url_predict_old = ''
# 定义要发送的数据
data_new = {
    "goods_a": "商品描述A",
    "goods_b": "商品描述B"
}
goods_statistics = []
for s,sc in zip(test_sample,test_sample_correct):
    goods_name = s[1].split.replace('对比相同和相似物资:','')
    goods_a = goods_name[0]
    goods_b = goods_name[1]
    data_new['good']
    print(s)
    print(sc)
    break

['913667', '对比相同和相似物资:阳语智能差压变送器（单法兰）YPDFN/250Kpa/衬PTFE/DN80/50mm/0m/0.5%/100℃/HART,阳语YPDFN/250Kpa/衬PTFE/DN80/50mm/0m/0.5%/100℃/HART智能差压变送器（单法兰）', '1', '1']
['913667', '对比相同和相似物资:阳语智能差压变送器（单法兰）YPDFN/250Kpa/衬PTFE/DN80/50mm/0m/0.5%/100℃/HART,阳语YPDFN/250Kpa/衬PTFE/DN80/50mm/0m/0.5%/100℃/HART智能差压变送器（单法兰）', '1', '1']


In [26]:
s[1].replace('对比相同和相似物资:','').split(',')

['阳语智能差压变送器（单法兰）YPDFN/250Kpa/衬PTFE/DN80/50mm/0m/0.5%/100℃/HART',
 '阳语YPDFN/250Kpa/衬PTFE/DN80/50mm/0m/0.5%/100℃/HART智能差压变送器（单法兰）']

# Predict

In [1]:
from pybert.model.bert_for_multi_label import BertForMultiLable
import torch
from transformers import BertTokenizer
from services.predict import data_predict

# 获取当前可用的CUDA设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForMultiLable.from_pretrained('T:\WorkSpace\\NLP\Bert-Multi-Label-Text-Classification\pybert\output\checkpoints\\bert', num_labels=2).to(device)

tokenizer = BertTokenizer('T:\WorkSpace\\NLP\Bert-Multi-Label-Text-Classification\pybert\pretrain\\bert\\bert-uncased\\vocab.txt', True)

E:\Anaconda\envs\Bert-Classification\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
examples, predict_result = data_predict(str_a='FORANT笔式pH测试仪871174370~14pH1台',
        str_b='BANTE/般特六合一水质检测仪M852-1~15pH1台',
        model=model,tokenizer=tokenizer,
        max_seq_len=512,example_type='predict',device=device)
print(predict_result)

tensor([[0.0160, 0.5531]], device='cuda:0')


In [3]:
from tqdm import tqdm

num_times = 1000

s = 's'
for i in tqdm(range(num_times)):
    # 发送POST请求并获取响应
   examples, predict_result = data_predict(str_a='FORANT笔式pH测试仪871174370~14pH1台'+i*s,
        str_b='BANTE/般特六合一水质检测仪M852-1~15pH1台',
        model=model,tokenizer=tokenizer,
        max_seq_len=512,example_type='predict',device=device)

100%|██████████| 1000/1000 [00:14<00:00, 67.82it/s]


In [ ]:
import requests
from tqdm import tqdm

num_times = 1000
# 修改为您的API地址，如果在本地运行，可以使用 http://127.0.0.1:8000/predict
url = "http://localhost:8000/predict"

# 定义要发送的数据
data = {
    "goods_a": "商品描述A",
    "goods_b": "商品描述B"
}

for i in tqdm(range(num_times)):
    # 发送POST请求并获取响应
    response = requests.post(url, json=data)